In [ ]:
#博客來新書(60天)
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from datetime import datetime
import re  # 用於提取 bookID

# 分類頁面基礎URL模板，n 為 01 到 21，包含頁數參數
base_url_template = "https://www.books.com.tw/web/books_nbtopm_{:02d}/?o=1&v=1&page={}"

# 動態生成分類頁面的鏈結，n 從 01 到 21
categories = [n for n in range(1, 22)]

# 生成當天日期並加入到 CSV 文件名中
current_date = datetime.now().strftime('%m%d')
output_file = rf'C:\Users\USER\Desktop\newarrive_books_data{current_date}.csv'

# CSV檔案的標題行，新增 'bookID'
csv_headers = ["bookID", "書名", "出版日期", "優惠價", "折扣", "價格", "連結"]

# 每頁的書籍數量閾值，當書籍數量少於100時，表示已到最後一頁
PAGE_BOOK_THRESHOLD = 100

# 抓取單個分類頁面的書籍資訊
def scrape_books(category_num, page_num):
    url = base_url_template.format(category_num, page_num)
    print(f"正在抓取分類 {category_num} 第 {page_num} 頁：{url}")
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # 檢查請求是否成功
        soup = BeautifulSoup(response.text, 'html.parser')
    
        # 找到頁面中的書籍條目
        book_items = soup.select('div.mod_a div.wrap div.item')
    
        if not book_items:  # 如果沒有書籍條目，說明已到最後一頁
            return None
    
        books_data = []
    
        for idx, item in enumerate(book_items, start=1):
            try:
                # 監控進度，告知處理哪本書
                print(f"正在處理分類 {category_num} 第 {page_num} 頁的第 {idx} 本書...")
                
                # 書名
                title_tag = item.select_one('h4 a')
                title = title_tag.text.strip() if title_tag else 'N/A'
                
                # 書籍鏈結
                book_link = title_tag['href'] if title_tag else 'N/A'
                full_link = book_link
                
                # 提取 bookID (即 URL 中的數字部分)
                book_id = re.search(r'/products/(\d+)', full_link).group(1) if re.search(r'/products/(\d+)', full_link) else 'N/A'
                
                # 出版日期
                pub_date_tag = item.select_one('ul.list li.info span')
                pub_date = pub_date_tag.text.split('：')[-1].strip() if pub_date_tag else 'N/A'
                
                # 優惠價
                price_section = item.select_one('ul.price li.set2')
                discount_tag = price_section.select_one('strong')  # 折扣
                price_tag = price_section.select('strong')[-1]  # 價格
                
                discount = discount_tag.text.strip() if discount_tag else 'N/A'
                price = price_tag.text.strip().replace('元', '') if price_tag else 'N/A'
                
                # 將 bookID 和其他數據一起保存
                books_data.append([book_id, title, pub_date, discount, price, full_link])
            
            except Exception as e:
                print(f"解析條目時出錯：{e}")
        
        return books_data
    except Exception as e:
        print(f"抓取分類 {category_num} 第 {page_num} 頁時發生錯誤：{e}")
        return None

# 保存數據到CSV文件
def save_to_csv(data, file_name):
    with open(file_name, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(csv_headers)
        writer.writerows(data)

# 主程式
def main():
    all_books_data = []
    
    for category_num in categories:
        page_num = 1
        while True:
            books_data = scrape_books(category_num, page_num)
            
            # 如果 books_data 是 None，說明該頁沒有書籍數據，退出該分類的翻頁
            if books_data is None:
                print(f"分類 {category_num} 已抓取完畢，總共抓取了 {page_num - 1} 頁")
                break  # 退出該分類的翻頁
            
            all_books_data.extend(books_data)
            
            # 監控進度，告知處理完哪一頁
            print(f"從分類 {category_num} 的第 {page_num} 頁抓取了 {len(books_data)} 本書籍")
            
            # 如果當前頁書籍數量少於100，說明已到最後一頁，跳到下一分類
            if len(books_data) < PAGE_BOOK_THRESHOLD:
                break
            
            # 繼續抓取下一頁
            page_num += 1
            time.sleep(2)  # 每頁之間休眠一段時間，避免伺服器過載
        
        # 每抓取完一個分類後，隨機暫停 20 到 180 秒
        pause_time = random.uniform(20, 180)
        print(f"分類 {category_num} 完成，共抓取了 {len(all_books_data)} 本書籍，暫停 {pause_time:.2f} 秒")
        time.sleep(pause_time)
    
    # 保存所有數據到CSV文件
    save_to_csv(all_books_data, output_file)
    print(f"數據已保存到 {output_file}")

# 執行主程式
if __name__ == "__main__":
    main()


正在抓取分類 1 第 1 頁：https://www.books.com.tw/web/books_nbtopm_01/?o=1&v=1&page=1
正在處理分類 1 第 1 頁的第 1 本書...
正在處理分類 1 第 1 頁的第 2 本書...
正在處理分類 1 第 1 頁的第 3 本書...
正在處理分類 1 第 1 頁的第 4 本書...
正在處理分類 1 第 1 頁的第 5 本書...
正在處理分類 1 第 1 頁的第 6 本書...
正在處理分類 1 第 1 頁的第 7 本書...
正在處理分類 1 第 1 頁的第 8 本書...
正在處理分類 1 第 1 頁的第 9 本書...
正在處理分類 1 第 1 頁的第 10 本書...
正在處理分類 1 第 1 頁的第 11 本書...
正在處理分類 1 第 1 頁的第 12 本書...
正在處理分類 1 第 1 頁的第 13 本書...
正在處理分類 1 第 1 頁的第 14 本書...
正在處理分類 1 第 1 頁的第 15 本書...
正在處理分類 1 第 1 頁的第 16 本書...
正在處理分類 1 第 1 頁的第 17 本書...
正在處理分類 1 第 1 頁的第 18 本書...
正在處理分類 1 第 1 頁的第 19 本書...
正在處理分類 1 第 1 頁的第 20 本書...
正在處理分類 1 第 1 頁的第 21 本書...
正在處理分類 1 第 1 頁的第 22 本書...
正在處理分類 1 第 1 頁的第 23 本書...
正在處理分類 1 第 1 頁的第 24 本書...
正在處理分類 1 第 1 頁的第 25 本書...
正在處理分類 1 第 1 頁的第 26 本書...
正在處理分類 1 第 1 頁的第 27 本書...
正在處理分類 1 第 1 頁的第 28 本書...
正在處理分類 1 第 1 頁的第 29 本書...
正在處理分類 1 第 1 頁的第 30 本書...
正在處理分類 1 第 1 頁的第 31 本書...
正在處理分類 1 第 1 頁的第 32 本書...
正在處理分類 1 第 1 頁的第 33 本書...
正在處理分類 1 第 1 頁的第 34 本書...
正在處理分類 1 第 1 頁的第 35 本書...
正在處理分類 1 第 1 頁的第 36 本書.

In [22]:
csv_file = r"C:\Users\USER\Desktop\newarrive_books_data1004.csv"

# 打開並讀取 CSV 文件
with open(csv_file, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    
    # 讀取 CSV 文件的每一行
    for row in csv_reader:
        print(row)

['bookID', '書名', '出版日期', '優惠價', '折扣', '價格', '連結']
['0011000997', '克莉絲蒂繁體中文版20週年紀念珍藏67-80', '2024-10-04', '7', '3682', 'https://www.books.com.tw/products/0011000997?loc=P_0005_001']
['0011001523', '前往天堂樂園【上、下冊】', '2024-10-04', '75', '712', 'https://www.books.com.tw/products/0011001523?loc=P_0005_002']
['0011001903', '長夜半生(精裝)', '2024-10-04', '79', '472', 'https://www.books.com.tw/products/0011001903?loc=P_0005_003']
['0011002155', '長生從煉丹宗師開始15', '2024-10-04', '79', '142', 'https://www.books.com.tw/products/0011002155?loc=P_0005_004']
['0011002156', '陣問長生17', '2024-10-04', '79', '142', 'https://www.books.com.tw/products/0011002156?loc=P_0005_005']
['0011002157', '高武紀元03', '2024-10-04', '79', '142', 'https://www.books.com.tw/products/0011002157?loc=P_0005_006']
['0011002161', '劍定乾坤62', '2024-10-04', '79', '142', 'https://www.books.com.tw/products/0011002161?loc=P_0005_007']
['0011002164', '龍虎道主47', '2024-10-04', '79', '142', 'https://www.books.com.tw/products/0011002164?loc=P_0005_008']
[

In [8]:
#每日新增測試
import requests
from bs4 import BeautifulSoup
import csv
import time
from datetime import datetime, timedelta
import random
import os

# 基本的 URL 模板，n 為分類號碼
base_url_template = 'https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page={}'
categories = [n for n in range(1, 22)]  # 模擬不同分類的頁數範圍

# 設定 60 天前的篩選條件
date_threshold = datetime.now() - timedelta(days=60)

# 獲取前一天的日期
yesterday_date = (datetime.now().date() - timedelta(days=1)).strftime('%m%d')
yesterday_csv_file_path = rf'C:\Users\USER\Desktop\prebooks_data{yesterday_date}.csv'

# 檢查昨天的 CSV 文件是否存在
if os.path.exists(yesterday_csv_file_path):
    with open(yesterday_csv_file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # 跳過表頭
        yesterday_books = [row[0] for row in reader]
else:
    yesterday_books = []

# CSV 文件的保存路徑，加入當天日期
current_date = datetime.now().strftime('%m%d')
csv_file_path = rf'C:\Users\USER\Desktop\prebooks_data{current_date}.csv'

# CSV檔案的標題行
csv_headers = ['書名', '作者', '出版日期', '折數', '優惠價', '鏈結']

# 保存數據到 CSV 文件
def save_to_csv(data, file_name):
    with open(file_name, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(csv_headers)
        writer.writerows(data)

# 抓取單個分類頁面的書籍資訊
def scrape_books(page_num):
    try:
        url = base_url_template.format(page_num)
        print(f"正在抓取第 {page_num} 頁：{url}")
        response = requests.get(url)
        response.raise_for_status()  # 檢查請求是否成功
        soup = BeautifulSoup(response.text, 'html.parser')
    
        # 找到頁面中的書籍條目
        book_items = soup.select('div.item')
    
        if not book_items:  # 如果沒有書籍條目，說明已到最後一頁
            return None
    
        books_data = []
        for item in book_items:
            try:
                # 書名
                title_tag = item.select_one('h4 a')
                title = title_tag.text.strip() if title_tag else 'N/A'
                
                # 書籍鏈結
                book_link = title_tag['href'] if title_tag else 'N/A'
                full_link = "https://www.books.com.tw" + book_link
                
                # 作者及出版日期
                info = item.select_one('li.info').text.strip()
                authors = info.split('，')[0]
                pub_date_str = info.split('出版日期：')[-1].strip()
                pub_date = datetime.strptime(pub_date_str, '%Y/%m/%d')
                
                # 優惠價和折數
                price_box = item.select_one('ul.price li.set2').text.strip()
                discount = price_box.split('折')[0].strip().replace('優惠價：', '')
                price = price_box.split('折')[-1].strip().replace('元', '').strip()
                
                # 如果出版日期在 60 天內，且今天未抓取過，才抓取數據
                if pub_date >= date_threshold and title not in yesterday_books:
                    books_data.append([title, authors, pub_date_str, f"{discount}折", f"{price}元", full_link])
            except Exception as e:
                print(f"解析條目時出錯：{e}")
        
        return books_data
    except Exception as e:
        print(f"抓取第 {page_num} 頁時發生錯誤：{e}")
        return None

# 主程式
def main():
    all_books_data = []

    for category_num in categories:
        page_num = 1
        while True:
            books_data = scrape_books(page_num)
            
            if not books_data:  # 如果沒有書籍數據，則跳出該分類
                break
            
            all_books_data.extend(books_data)
            print(f"分類 {category_num} 的第 {page_num} 頁抓取到 {len(books_data)} 本書籍")
            
            page_num += 1
            time.sleep(2)  # 在每頁之間暫停 2 秒

        # 每個分類結束後暫停 60 到 180 秒
        wait_time = random.randint(60, 180)
        print(f"分類 {category_num} 抓取完畢，休息 {wait_time} 秒")
        time.sleep(wait_time)

    # 保存抓取的數據到 CSV 文件
    save_to_csv(all_books_data, csv_file_path)
    print(f"新增書籍資訊已保存到 {csv_file_path}")

# 執行主程式
if __name__ == "__main__":
    main()



正在抓取第 1 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=1
分類 1 的第 1 頁抓取到 7 本書籍
正在抓取第 2 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=2
分類 1 的第 2 頁抓取到 7 本書籍
正在抓取第 3 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=3
分類 1 的第 3 頁抓取到 7 本書籍
正在抓取第 4 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=4
分類 1 的第 4 頁抓取到 7 本書籍
正在抓取第 5 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=5
分類 1 的第 5 頁抓取到 7 本書籍
正在抓取第 6 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=6
分類 1 的第 6 頁抓取到 7 本書籍
正在抓取第 7 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=7
分類 1 的第 7 頁抓取到 7 本書籍
正在抓取第 8 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=8
分類 1 的第 8 頁抓取到 7 本書籍
正在抓取第 9 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=9
分類 1 的第 9 頁抓取到 7 本書籍
正在抓取第 10 頁：https://www.books.com.tw/web/sys_prebooks/books/?o=1&v=1&page=10
分類 1 的第 10 頁抓取到 7 本書籍
正在抓取第 11 頁：https://www.books.com.tw/web/sys_pre

KeyboardInterrupt: 